In [ ]:
#https://nlpforhackers.io/wordnet-sentence-similarity/
#https://realpython.com/python-csv/
#https://chrisalbon.com/#python
import re
import csv
import json
import bisect
import gensim
import nltk
from multiprocessing.pool import ThreadPool
import numpy as np
import pandas as pd
import itertools
import scipy.interpolate as interpolate
from operator import itemgetter
from scipy import stats
from nltk import pos_tag
from decimal import Decimal
from IPython.display import clear_output
folder_path = 'D:/MS CS/RS DATA/dataset/WMIRank/'
print("success")

#### cac_k_l -> number of co-authored citations of papers for Author L and K
#### tac_k -> total numbers of citations of authors K


#### fap_k -> common papers written by K as 1st author
#### fap_l -> common papers written by L as 1st author
#### tap_k -> total papers written by author K

In [ ]:
author_to_paper = pd.read_json(folder_path+'author-2-paper.json', orient='records', encoding='utf8', lines=True)
common_paper = pd.merge(author_to_paper, author_to_paper, on=['pid'], how='inner', suffixes=("l", "k"))
distinct_common_paper = common_paper[common_paper.aidk != common_paper.aidl]

rs_corpus = pd.read_json(folder_path+'rs-corpus.json', orient='records', encoding='utf8', lines=True)
author_corpus = pd.read_json(folder_path+'author-corpus.json', orient='records', encoding='utf8', lines=True)
print("success")

In [ ]:
wmi_score = []

for k_aidk, v_aidk in distinct_common_paper.groupby("aidk"):
    clear_output()
    print(k_aidk)

    # Author’s Papers & Citations.
    tac_k = rs_corpus[rs_corpus.id.isin(v_aidk['pid'])]['citation'].sum()
    
    # Authors’ Papers & Order of Appearance.
    fap_k = v_aidk[v_aidk.posk==1]['posk'].count()
    tap_k = v_aidk['posk'].nunique()
    
    # Author papers and their corresponding Venue’ average citations.
    tvc_k = rs_corpus[rs_corpus.id.isin(v_aidk['pid'])]['vas'].sum()

    for k_aidl, v_aidl in v_aidk.groupby("aidl"):
        
        # Co-Author Citations based Mutual Influence
        CACWI = Decimal((rs_corpus[rs_corpus.id.isin(v_aidl['pid'])]['citation'].sum())/tac_k)
        
        # Co-Author Order based Mutual Influence
        CAOWI = Decimal(((v_aidl[v_aidl.posl==1]['posl'].count())+fap_k)/(fap_k+tap_k))
        
        # Co-Author Venue’s Citations Based Mutual Influence
        CAVWI = Decimal((rs_corpus[rs_corpus.id.isin(v_aidl['pid'])]['vas'].sum())/tvc_k)
            
        wmi_score.append([k_aidl, k_aidk, CACWI, CAOWI, CAVWI])
        
print("saving...")
pd.DataFrame(wmi_score).to_csv(folder_path+'wmi-score.csv', sep=',', encoding='utf-8', header=None, index=False)
print("success")

In [ ]:
wmi_score = pd.read_csv(folder_path+'wmi-score.csv', names=['aidi', 'aidj', 'CACWI', 'CAOWI', 'CAVWI'])
group_by_aidi_wmi_score = wmi_score.groupby("aidi")
author_corpus = pd.read_json(folder_path+'author-corpus.json', orient='records', encoding='utf8', lines=True)
author_id = author_corpus['aid'].unique().tolist()
print("sucess")

In [ ]:
n = len(author_id)
d = 0.85
wmi_rank = []

for v_aidi in author_id:
    
    clear_output()
    print(v_aidi)
    T_score = 0.0
    vocabulary_co_author = group_by_aidi_wmi_score.get_group(v_aidi)
    for index, v_aidj in vocabulary_co_author.iterrows():
        
        upper_T = (float(v_aidj.CACWI)*float(v_aidj.CAOWI)*float(v_aidj.CAVWI))
        lower_T = 0.0
        for index_k , v_aidk in vocabulary_co_author.iterrows():
            
            lower_row = wmi_score[(wmi_score.aidi == v_aidk.aidi) & (wmi_score.aidj == v_aidj.aidj)]
            lower_T = lower_T + (float(lower_row.CACWI)*float(lower_row.CAOWI)*float(lower_row.CAVWI))
            
        T_score = T_score + (upper_T/(lower_T+1))
    
    #WMIRank = (((1-d)/n) + (d*T_score))
    wmi_rank.append([v_aidi, (((1-d)/n) + (d*T_score))])
    
print("saving")
pd.DataFrame(wmi_rank).to_csv(folder_path+'wmi-rank-85.csv', sep=',', encoding='utf-8', header=None, index=False)
print("success")

In [ ]:
wmi_rank = pd.read_csv(folder_path+'wmi-rank-85.csv', names=['aid', 'score'])
wmi_rank.sort_values(by=['score'], inplace=True, ascending=False)
wmi_rank.head()

In [ ]:
author_ids = []
author_corpus = pd.read_json(folder_path+'author-corpus.json', orient='records', encoding='utf8', lines=True)
for ind, author in author_corpus.iterrows():
    wmi_count = wmi_score[wmi_score.aidi==author.aid]['aidi'].nunique()
    if wmi_count == 0:
        author_ids.append(author.aid)
        
author_corpus = author_corpus[~author_corpus['aid'].isin(author_ids)]
author_corpus.to_json(folder_path+'author-corpus.json', orient='records', lines=True)

In [ ]:
wmi_score = pd.read_csv(folder_path+'wmi-rank-85.csv', names=['aid', 'score'])
with open(folder_path+'author-corpus.json', 'r', encoding='utf8') as f:
    author_corpus = []
    for doc in f:
        data = json.loads(doc)
        if (data['n_citation']/data['n_paper']) > 15:
            data['score'] = float(wmi_score[wmi_score.aid == data['aid']].score)*2
        else:
            data['score'] = float(wmi_score[wmi_score.aid == data['aid']].score)
        del data['p_index']
        del data['pa_index']
        author_corpus.append(data)
    f.close()
author_corpus = pd.DataFrame(author_corpus)
author_corpus.to_json(folder_path+'wmi-author-corpus.json', orient='records', lines=True)
print("success")

In [ ]:
author_corpus = pd.read_json(folder_path+'wmi-author-corpus.json', orient='records', encoding='utf8', lines=True)
author_corpus.sort_values(by=['score'], inplace=True, ascending=False)
author_corpus = author_corpus.reset_index(drop=True)

hs_score = author_corpus.score.tolist()
minScore = min(hs_score)
maxScore = max(hs_score)
author_corpus_lst = []
for k, data in author_corpus.iterrows():
    data['n_score'] = (data.score-minScore)/(maxScore-minScore)
    author_corpus_lst.append(data)

pd.DataFrame(author_corpus_lst).to_json(folder_path+'wmi-author-corpus.json', orient='records', lines=True)
print("success")